In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

import numpy as np
import sklearn
from sklearn import metrics
import nltk 
from collections import Counter
import matplotlib.pyplot as plt
import itertools
import ast

nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))


/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /lfs/1/simran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /lfs/1/simran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/notebooks')
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/')
import utils_not_for_public as du


es, esp, type_vocab, typeid2typename, type_vocab_wd, typeid2typename_wd, alias_qid_count = du.load_esp("wiki_0905", overwrite=False)     


Reading in existing entity word bags from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_bag_of_words
Reading in vocab from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_all_words/all_words_vocab.marisa
Loaded entity symbols.
Loading alias qid counts from /dfs/scratch0/lorr1/data_prep/data/wiki0905/stats/alias_qid_traindata_withaugment.json
Found 506 hyena types
Found 26933 wikidata types
FINISHED LOADING IN 84.93753552436829


In [25]:
esp = esp_0905

In [5]:
def load_rel_names_0905():
    return json.load(open("/dfs/scratch1/simran/bootleg_downstream/archive/wikidata_mappings/pid_names.json"))
import ujson
import json
def load_json_file(filename):
    with open(filename, 'r') as f:
        contents = ujson.load(f)
    return contents


typename2title_wd_0905 = load_json_file("/dfs/scratch0/lorr1/bootleg/embs/wikidatatypeid_to_title_0905.json")
typeqid_to_id_0905 = json.load(open("/dfs/scratch0/lorr1/bootleg/embs/wikidata_to_typeid_0905.json"))
rel2id_0905 = '/dfs/scratch0/lorr1/bootleg/embs/relation_to_typeid_0905.json'
rel_to_name_0905 = load_rel_names_0905()

In [57]:
import json
relations_file = json.load(open('/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/bootleg_downloads/emb_data/kg_relation_types_0905.json'))
types_file = json.load(open('/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/bootleg_downloads/emb_data/wikidata_types_0905.json'))

In [7]:
# Bootleg utility functions:
# BY ALIAS: 
def get_candidates(alias):
    try:
        # To get qid candidates of an alias
        cands = esp.get_qid_cands(alias)
        return cands
    except:
        cands = esp.get_qid_cands(alias)
    
# get_candidates('distortion')

In [19]:
# def get_related_entities(qid):
#     # Get all connected qids for a give qid
#     related_qids = du.get_related_qids(qid, esp)
#     # print(f"Related QIDs {related_qids}")
#     return related_qids, [es.get_title(r) for r in related_qids]

# #get_related_entities('Q179202')

In [20]:
# def get_relations(qid, qid2):
#     rels = [esp.get_relation_name(r) for r in esp.get_all_relations(qid, qid2)]
#     rels = [rel_to_name.get(rel, rel) for rel in rels]
#     print(f"Rels {rels}")
    
# get_relations('Q179202','Q30')

Rels ['country of citizenship']


In [14]:
# Manual User Inputs
model = '/dfs/scratch1/simran/bootleg_downstream/error_outputs/baseline_bootleg_test_rev_ent.csv' #baseline spanbert without bootleg
spanbert = '/dfs/scratch1/simran/bootleg_downstream/error_outputs/predictionsspanbert_test_0908.csv'
spanboot = '/dfs/scratch1/simran/bootleg_downstream/error_analysis_code/ft_boot_101520/test_predictions.csv'

In [9]:
LABEL_TO_ID = {'no_relation': 0, 'per:title': 1, 'org:top_members/employees': 2, 'per:employee_of': 3, 
               'org:alternate_names': 4, 'org:country_of_headquarters': 5, 'per:countries_of_residence': 6, 
               'org:city_of_headquarters': 7, 'per:cities_of_residence': 8, 'per:age': 9, 
               'per:stateorprovinces_of_residence': 10, 'per:origin': 11, 'org:subsidiaries': 12, 
               'org:parents': 13, 'per:spouse': 14, 'org:stateorprovince_of_headquarters': 15, 'per:children': 16, 
               'per:other_family': 17, 'per:alternate_names': 18, 'org:members': 19, 'per:siblings': 20, 
               'per:schools_attended': 21, 'per:parents': 22, 'per:date_of_death': 23, 'org:member_of': 24, 
               'org:founded_by': 25, 'org:website': 26, 'per:cause_of_death': 27, 
               'org:political/religious_affiliation': 28, 'org:founded': 29, 'per:city_of_death': 30, 
               'org:shareholders': 31, 'org:number_of_employees/members': 32, 'per:date_of_birth': 33, 
               'per:city_of_birth': 34, 'per:charges': 35, 'per:stateorprovince_of_death': 36, 'per:religion': 37, 
               'per:stateorprovince_of_birth': 38, 'per:country_of_birth': 39, 'org:dissolved': 40, 
               'per:country_of_death': 41}

LABEL_LST = list(LABEL_TO_ID.keys())

STANFORD_NER_TYPES = ['DATE', 'LOCATION', 'MONEY', 'ORGANIZATION', 'PERCENT', 'PERSON', 'TIME']
punctuation = [',', ':', '.', ';', "'", ')', '(', "'s", '--', '-', '``', "''"]

In [10]:
# Utility functions: accepts a df with corrected slices, and outputs the predicted result
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support as score
def score_corrections(df, pred_name):
    import scorer
    scorer.score(df['relation'].tolist(), df[pred_name].tolist(), verbose=True)
    df_errs = df[df['relation'] != df[pred_name]]
    print("Number of Errors: ", df_errs.shape[0])
# score_corrections(df_results)

In [11]:
# basic error rate by relation 
def error_rate_by_relation(df_r, key, df_e):
    relation_df = pd.DataFrame(columns=['relation','error_rate','error_count','total_count'])
    index = 0
    for k, v in LABEL_TO_ID.items():
        df_relation_tot = df_r[df_r[key] == k] # all rows with k = true trelation
        tot = df_relation_tot.shape[0] # number of examples with this true relation
    
        df_relation_err = df_e[df_e[key] == k] # error rows with k = true trelation
        err = df_relation_err.shape[0] # number of errors with this true relation
        error_rate = err/tot
    
        relation_df.loc[index] = pd.Series({'relation':k, 'error_rate':error_rate, 'error_count':err, 'total_count':tot})
        index += 1
    
    print(relation_df.sort_values('relation'))

In [15]:
# Load the model data 
df_results = pd.read_csv(model)
df_results.rename(columns={'prediction':'prediction_model'}, inplace=True)

# Load the spanbert data
df_results_spanbert = pd.read_csv(spanbert)
map_id_pred = {}
for ind, row in df_results_spanbert.iterrows():
    map_id_pred[row['id']] = row['prediction']
df_results['prediction_spanbert'] = df_results['id'].map(map_id_pred)
for key in df_results.columns.values:
    if key != 'prediction':
        df_results_spanbert[key] = df_results[key]
        
        
# Load the spanboot data
df_results_spanboot = pd.read_csv(spanboot)
map_id_pred = {}
for ind, row in df_results_spanboot.iterrows():
    map_id_pred[row['id']] = row['prediction']
df_results['prediction_spanboot'] = df_results['id'].map(map_id_pred)
for key in df_results.columns.values:
    if key != 'prediction':
        df_results_spanboot[key] = df_results[key]
        
# error rates
df_errors = df_results[df_results['relation'] != df_results['prediction_model']]
df_errors_spanbert = df_results[df_results['relation'] != df_results['prediction_spanbert']]
df_errors_spanboot = df_results[df_results['relation'] != df_results['prediction_spanboot']]


print("FULL_model size: ", df_results.shape)
print("ERRS_model size: {}\n".format(df_errors.shape))

print("FULL_baseline size: ", df_results_spanbert.shape)
print("ERRS_baseline size: {}\n".format(df_errors_spanbert.shape))

print("FULL_spanboot size: ", df_results_spanboot.shape)
print("ERRS_spanboot size: ", df_errors_spanboot.shape)



FULL_model size:  (15509, 28)
ERRS_model size: (1642, 28)

FULL_baseline size:  (15509, 28)
ERRS_baseline size: (1346, 28)

FULL_spanboot size:  (15509, 29)
ERRS_spanboot size:  (1129, 28)


In [16]:
print(df_results.columns.values)

['obj' 'subj' 'prop_ner' 'obj_pos' 'subj_ner' 'obj_mentions' 'subj_type'
 'obj_ner' 'id' 'subj_pos' 'prediction_model' 'obj_qids' 'example' 'qids'
 'subj_qids' 'prop_mentions' 'num_ner' 'obj_type' 'obj_leng' 'subj_leng'
 'relation' 'real_mentions' 'stanford_ner' 'subj_mentions'
 'separation_dist' 'mentions' 'prediction_spanbert' 'prediction_spanboot']


## Analysis by Relation

In [17]:
#error_rate_by_relation(df_results, 'relation_model', df_errors)
relation_df = pd.DataFrame(columns=['relation','errrate_span','errcount_span', 'errrate_boot','errcount_boot','total_count'])
index = 0
for k, v in LABEL_TO_ID.items():
    if df_results_spanbert[df_results_spanbert['relation'] == k].shape[0] > 0:
        df_relation_tot_span = df_results_spanbert[df_results_spanbert['relation'] == k] # all rows with k = true trelation
        tot_span = df_relation_tot_span.shape[0] # number of examples with this true relation
    
        df_relation_err_span = df_errors_spanbert[df_errors_spanbert['relation'] == k] # error rows with k = true trelation
        err_span = df_relation_err_span.shape[0] # number of errors with this true relation
        error_rate_span = err_span/tot_span
        
        df_relation_tot_boot = df_results_spanboot[df_results_spanboot['relation'] == k] # all rows with k = true trelation
        tot_boot = df_relation_tot_boot.shape[0] # number of examples with this true relation
    
        df_relation_err_boot = df_errors_spanboot[df_errors_spanboot['relation'] == k] # error rows with k = true trelation
        err_boot = df_relation_err_boot.shape[0] # number of errors with this true relation
        error_rate_boot = err_boot/tot_boot
    
        relation_df.loc[index] = pd.Series({'relation':k, 'errrate_span':error_rate_span, 'errcount_span':err_span, 'errrate_boot':error_rate_boot,'errcount_boot': err_boot, 'total_count':tot_span})
        index += 1
    
print(relation_df.sort_values('relation'))

                               relation  errrate_span errcount_span  \
0   no_relation                          0.056838      704            
4   org:alternate_names                  0.106122      26             
7   org:city_of_headquarters             0.191011      17             
5   org:country_of_headquarters          0.478261      44             
39  org:dissolved                        1.000000      1              
29  org:founded                          0.027027      1              
25  org:founded_by                       0.144737      11             
24  org:member_of                        0.750000      3              
19  org:members                          0.437500      7              
32  org:number_of_employees/members      0.214286      3              
13  org:parents                          0.508772      29             
28  org:political/religious_affiliation  0.200000      2              
31  org:shareholders                     0.666667      2              
15  or

In [18]:
# get_related_entities('Q179202')
#get_relations('Q179202', 'Q30')

In [19]:
sub_df = df_errors_spanboot[df_errors_spanboot['relation'] == 'per:countries_of_residence']
for ind, row in sub_df.iterrows():
    print(row[cols])
    print()

NameError: name 'cols' is not defined

# Compare Model vs. Baseline Errors

We want to confirm that the improvement ties back to insights in bootleg - i.e., the mentions/types/relations in bootleg's database.

In [42]:
# get the set difference of errors
df_spanbert_only_errors = df_errors_spanbert[~df_errors_spanbert['id'].isin(df_errors_spanboot['id'])]
df_spanboot_only_errors = df_errors_spanboot[~df_errors_spanboot['id'].isin(df_errors_spanbert['id'])]
print("The #examples our model improves on vs. baseline are: ", df_spanbert_only_errors.shape[0])
print("The #examples our model does worse on vs. baseline are: ", df_spanboot_only_errors.shape[0])

# for i, row in df_spanbert_only_errors.iterrows():
#     if row['relation'] != "no_relation":
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_spanboot', 'prediction_spanbert', 'mentions', 'stanford_ner']])
#         print()
#     if i == 10:
#         break

The #examples our model improves on vs. baseline are:  664
The #examples our model does worse on vs. baseline are:  442


In [43]:
# THE DISTRIBUTION of RELATIONS WHEN MODEL IMPOVES OVER SPANBERT
print("THE DISTRIBUTION of RELATIONS WHEN MODEL IMPROVES OVER SPANBERT")
relations_errs_matchqids = []
for index, row in df_spanbert_only_errors.iterrows():
    relations_errs_matchqids.append(row['relation'])
print(Counter(relations_errs_matchqids))
print()

# THE DISTRIBUTION OF RELATIONS WHEN SPANBERT IMPROVES OVER MODEL
print(" THE DISTRIBUTION OF RELATIONS WHEN SPANBERT IMPROVES OVER MODEL")
relations_errs_matchqids = []
for index, row in df_spanboot_only_errors.iterrows():
    relations_errs_matchqids.append(row['relation'])
print(Counter(relations_errs_matchqids))


THE DISTRIBUTION of RELATIONS WHEN MODEL IMPROVES OVER SPANBERT
Counter({'no_relation': 427, 'per:employee_of': 21, 'per:title': 21, 'per:age': 19, 'org:country_of_headquarters': 17, 'per:cities_of_residence': 15, 'per:charges': 14, 'org:stateorprovince_of_headquarters': 13, 'per:countries_of_residence': 13, 'org:alternate_names': 12, 'org:city_of_headquarters': 10, 'per:cause_of_death': 8, 'org:top_members/employees': 8, 'per:origin': 7, 'per:date_of_death': 7, 'per:stateorprovinces_of_residence': 7, 'org:founded_by': 6, 'per:siblings': 6, 'per:schools_attended': 5, 'org:subsidiaries': 5, 'per:children': 4, 'org:parents': 4, 'per:spouse': 4, 'per:parents': 3, 'org:number_of_employees/members': 2, 'per:city_of_death': 2, 'per:stateorprovince_of_death': 2, 'per:other_family': 1, 'per:stateorprovince_of_birth': 1})

 THE DISTRIBUTION OF RELATIONS WHEN SPANBERT IMPROVES OVER MODEL
Counter({'no_relation': 231, 'org:top_members/employees': 40, 'per:countries_of_residence': 22, 'per:title': 

### SPANBERT WORSE THAN BOOTLEG

Here we find that the *number* of proper nouns that have errors and receive bootleg mentions are about the same w/ vs. w/out using bootleg entity embeddings. 

In [23]:
# some constants
proper_noun = ['NNP', 'NNPS']
nonproper_noun = ['NN', 'NNS']
number_pos = ['CD']
nomention = ["['UNK']", "['UNK', 'UNK']", "['UNK', 'UNK', 'UNK']", "['UNK', 'UNK', 'UNK', 'UNK']", 
             "['UNK', 'UNK', 'UNK', 'UNK', 'UNK']"]
cols = ['example', 'relation','prediction_model', 'prediction_spanbert', 'prediction_spanboot', 'mentions', 'subj_pos', 'obj_pos', 'subj', 'obj', 'subj_mentions', 'subj_qids', 'obj_mentions', 'obj_qids', 'id']

In [21]:
rel_to_name = load_entity_profiles.load_rel_names()
def get_relations(subj_qids_str, obj_qids_str):
    subj_qids = ast.literal_eval(subj_qids_str)
    obj_qids = ast.literal_eval(obj_qids_str)
    subj_qid = subj_qids[0]
    obj_qid = obj_qids[0]
    
    rels = [esp.get_relation_name(r) for r in esp.get_all_relations(subj_qid, obj_qid)]
    rels = [rel_to_name.get(rel, rel) for rel in rels]
    return rels

In [22]:
def get_related_entities(qid):
    # Get all connected qids for a given qid
    related_qids = du.get_related_qids(qid, es)
    print(f"Related QIDs {related_qids}")
    print([es.get_title(qid) for qid in related_qids])

    lst = [es.get_title(qid) for qid in related_qids]

In [47]:
lst_pos_subj_spanbert = []
lst_subj_nomention_spanbert = {}
lst_pos_obj_spanbert = []
lst_obj_nomention_spanbert = {}

sub_df = df_spanbert_only_errors[cols]
for index, row in sub_df.iterrows():
    if any(pos in row['subj_pos'] for pos in proper_noun): # is the subj a proper noun?
        lst_pos_subj_spanbert.append(row['subj'])
        if any(null in row['subj_mentions'] for null in nomention): # is the bootleg mention empty for this proper noun?
            lst_subj_nomention_spanbert[row['id']] = row['subj']
            
    if any(pos in row['obj_pos'] for pos in proper_noun):
        lst_pos_obj_spanbert.append(row['obj'])
        if any(null in row['obj_mentions'] for null in nomention):
            lst_obj_nomention_spanbert[row['id']] = row['obj']
        
print("The number of proper noun subj in spanbert errors are:", len(lst_pos_subj_spanbert), "and obj are:", len(lst_pos_obj_spanbert))
print("The number of proper noun subj in spanbert errors that don't get a bootleg mention are:", len(lst_subj_nomention_spanbert.keys()), "and obj are:", len(lst_obj_nomention_spanbert.keys()))
print()

lst_pos_subj_spanbert.extend(lst_pos_obj_spanbert)
print("SPANBERT DOES WORSE THAN BOOTLEG MODEL ON THESE SUBJ/OBJ PROPER NOUNS, THIS MANY TIMES:")#print(len(lst_pos_subj_spanbert))
worst_for_spanbert = Counter(lst_pos_subj_spanbert).most_common(50)
print(worst_for_spanbert)
worst_for_spanbert = [tup[0] for tup in worst_for_spanbert]


The number of proper noun subj in spanbert errors are: 514 and obj are: 457
The number of proper noun subj in spanbert errors that don't get a bootleg mention are: 34 and obj are: 47

SPANBERT DOES WORSE THAN BOOTLEG MODEL ON THESE SUBJ/OBJ PROPER NOUNS, THIS MANY TIMES:
[("['corporate', 'library']", 25), ("['norris', 'church']", 24), ("['alico']", 18), ("['china']", 17), ("['national', 'restaurant', 'association']", 15), ("['tpi']", 13), ("['national', 'development', 'and', 'reform', 'commission']", 12), ("['ruben', 'van', 'assouw']", 12), ("['charles', 'ble', 'goude']", 12), ("['wen', 'qiang']", 11), ("['xie', 'caiping']", 11), ("['countrywide']", 11), ("['pda']", 10), ("['national', 'congress', 'of', 'american', 'indians']", 10), ("['national', 'urban', 'league']", 10), ("['ndrc']", 9), ("['chongqing']", 9), ("['nir', 'rosen']", 8), ("['alessi']", 8), ("['us']", 8), ("['u.s.']", 8), ("['progressive', 'democrats', 'of', 'america']", 7), ("['john', 'graham']", 7), ("['alan', 'gross']"

In [48]:
# Inspecting examples where bootleg had any relation for the subj, obj pair

count = 0
for index, row in df_spanbert_only_errors.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count += 1
print("For ", count, " spanbert only errors, bootleg has *some* relation between the subj and obj")

count = 0
for index, row in df_spanbert_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and not any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_spanboot', 'prediction_spanbert', 'mentions', 'stanford_ner']])
#         print(f"Rels {rels}")
#         print()
        count += 1

print("For ", count, " spanbert only errors, the existing bootleg relation is NOT a subset of the gold relation")


For  58  spanbert only errors, bootleg has *some* relation between the subj and obj
For  53  spanbert only errors, the existing bootleg relation is NOT a subset of the gold relation


In [49]:
# alternate names
alternate_name_errs = df_spanbert_only_errors[df_spanbert_only_errors['relation'].str.contains("alternate")].shape
print("For ", alternate_name_errs[0], " spanbert only errors, the gold label is ALTERNATE NAMES ")


For  12  spanbert only errors, the gold label is ALTERNATE NAMES 


In [50]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanbert_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1
        
print("For ", count, " spanbert only errors, the bootleg relation is a subset (or the same) as the gold relation")


For  5  spanbert only errors, the bootleg relation is a subset (or the same) as the gold relation


In [51]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanbert_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['prediction_spanbert']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1
        
print("For ", count, " spanbert errors, the bootleg relation is a subset (or the same) as the spanbert prediction")


For  7  spanbert errors, the bootleg relation is a subset (or the same) as the spanbert prediction


### FOR EXAMPLES WHERE SPANBERT IS BETTER THAN BOOTLEG

In [52]:
lst_pos_subj_model = []
lst_subj_nomention_model = {}
lst_pos_obj_model = []
lst_obj_nomention_model= {}

sub_df = df_spanboot_only_errors[cols]
for index, row in sub_df.iterrows():
    if any(pos in row['subj_pos'] for pos in proper_noun): # is the subj a proper noun?
        lst_pos_subj_model.append(row['subj'])
        if any(null in row['subj_mentions'] for null in nomention): # is the bootleg mention empty for this proper noun?
            lst_subj_nomention_model[row['id']] = row['subj']
            
    if any(pos in row['obj_pos'] for pos in proper_noun):
        lst_pos_obj_model.append(row['obj'])
        if any(null in row['obj_mentions'] for null in nomention):
            lst_obj_nomention_model[row['id']] = row['obj']
        
print("The number of proper noun subj in bootleg (&not spanbert) errors are:", len(lst_pos_subj_spanbert), "and obj are:", len(lst_pos_obj_spanbert))
print("The number of proper noun subj in bootleg (&not spanbert) errors that don't get a bootleg mention are:", len(lst_subj_nomention_spanbert.keys()), "and obj are:", len(lst_obj_nomention_spanbert.keys()))
print()

lst_pos_subj_model.extend(lst_pos_obj_model)
print("SPANBERT DOES BETTER THAN BOOTLEG MODEL ON THESE SUBJ/OBJ PROPER NOUNS")#print(len(lst_pos_subj_spanbert))
worst_for_model = Counter(lst_pos_subj_model).most_common(50)
print(worst_for_model)
worst_for_model = [tup[0] for tup in worst_for_model]

The number of proper noun subj in bootleg (&not spanbert) errors are: 971 and obj are: 457
The number of proper noun subj in bootleg (&not spanbert) errors that don't get a bootleg mention are: 34 and obj are: 47

SPANBERT DOES BETTER THAN BOOTLEG MODEL ON THESE SUBJ/OBJ PROPER NOUNS
[("['corporate', 'library']", 19), ("['ruben', 'van', 'assouw']", 14), ("['paul', 'hodgson']", 11), ("['alico']", 10), ("['matteo', 'messina', 'denaro']", 10), ("['eliza', 'samudio']", 9), ("['samudio']", 8), ("['xie', 'caiping']", 8), ("['us']", 8), ("['ahmed', 'rashid']", 7), ("['alan', 'gross']", 7), ("['national', 'congress', 'of', 'american', 'indians']", 6), ("['national', 'restaurant', 'association']", 6), ("['chen', 'tao']", 6), ("['wen', 'qiang']", 6), ("['ruben']", 6), ("['china']", 6), ("['cuba']", 6), ("['adela', 'dworin']", 5), ("['meredith', 'kercher']", 5), ("['tamaihia', 'lynae', 'moore']", 5), ("['norris', 'church', 'mailer']", 5), ("['tpi']", 5), ("['asia', 'bibi']", 5), ("['gross']", 5),

In [53]:
for ind, row in df_spanboot_only_errors.iterrows():
    if 'library' in row['subj']:
        print(row[cols])

example                `` what happened to their investments was of no interest to them , because they would already be paid , '' said paul hodgson , senior research associate at the corporate library , a shareholder activist group .                                                                                                                                 
relation               org:top_members/employees                                                                                                                                                                                                                                                                                                                          
prediction_model       org:top_members/employees                                                                                                                                                                                                                                  

In [54]:
# model_worse_than_spanbert = [tup for tup in worst_for_model if tup not in worst_for_spanbert]
# for item in model_worse_than_spanbert:
#     item = ast.literal_eval(item)
#     item = ' '.join(item)
#     cands = get_candidates(item)
#     print("Cands for ", item, " are: ", f"Cands {cands}", "\n")

In [55]:
sub_df = df_spanboot_only_errors[cols] #BOOTLEG ERRORS, and NOT BASELINE ERRORS
count_missed_propernoun = 0
count_missed_propernoun_with_bootrels = 0
ids_missed_propernoun_with_bootrels = []
for index, row in sub_df.iterrows():
    if any(pos in row['subj_pos'] for pos in proper_noun) or any(pos in row['obj_pos'] for pos in proper_noun):
#        print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
        rels = get_relations(row['subj_qids'], row['obj_qids'])
        if rels:
            count_missed_propernoun_with_bootrels += 1
            ids_missed_propernoun_with_bootrels.append(row['id'])
#         print(f"Rels {rels}")
#         print()
        count_missed_propernoun += 1
        
print("Number of examples where bootleg had a relation for the subj, obj pair: ", count_missed_propernoun_with_bootrels)
print("Number of examples where subj and/or obj is a proper noun: ", count_missed_propernoun)


Number of examples where bootleg had a relation for the subj, obj pair:  35
Number of examples where subj and/or obj is a proper noun:  391


In [56]:
# Inspecting examples where bootleg had any relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanboot_only_errors.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count += 1
print("For ", count, " bootleg errors, bootleg has *some* relation between the subj and obj\n")


print("For these bootleg errors, the existing bootleg relation is NOT a string-subset of the gold relation")
count = 0
for index, row in df_spanboot_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and not any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1

print("For ", count, " bootleg errors, the existing bootleg relation is NOT a subset of the gold relation")


For  35  bootleg errors, bootleg has *some* relation between the subj and obj

For these bootleg errors, the existing bootleg relation is NOT a string-subset of the gold relation
For  34  bootleg errors, the existing bootleg relation is NOT a subset of the gold relation


In [57]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_spanboot_only_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['relation']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1
        
print("For ", count, " bootleg errors, the bootleg relation is a subset (or the same) as the gold relation")


For  1  bootleg errors, the bootleg relation is a subset (or the same) as the gold relation


In [58]:
# Inspecting examples where bootleg had a relation for the subj, obj pair
# booterrors_with_relation_df = df_errors[df_errors['id'].isin(ids_missed_propernoun_with_bootrels)]
count = 0
for index, row in df_errors.iterrows():
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    if rels and any(rel for rel in rels if rel in row['prediction_model']):
#         print(row[['example', 'subj', 'obj', 'subj_qids', 'obj_qids', 'relation', 'prediction_model', 'prediction_spanbert', 'mentions']])
#         print(f"Rels {rels}")
#         print()
        count += 1
        
print("For ", count, " bootleg errors, the bootleg relation is a subset (or the same) as the bootleg prediction")


For  2  bootleg errors, the bootleg relation is a subset (or the same) as the bootleg prediction


#### For these proper nouns that don't get a bootleg mention, is it because they are not in bootleg's database?

In [59]:
#print(lst_subj_nomention_model)

no_candidates_values = []
has_candidates_values = []
# alias is the subj/obj that did not receive a bootleg mention
for idx, value in lst_subj_nomention_model.items():
    row = df_results[df_results['id'] == idx]   
    value = ast.literal_eval(value)
    alias = " ".join(value)
    try:
        cands = esp.get_qid_cands(alias)
        #print("{}\n{}\n{}\n\n".format(row['example'], alias, f"Cands {cands}") # uncomment to view the alias in context downstream
        has_candidates_values.append(alias)
    except: 
        no_candidates_values.append(alias)

print("Of these values that received no bootleg mention,", len(has_candidates_values), "(total)/", len(set(has_candidates_values)), "(unique), ARE IN the bootleg database.")

######## THIS SET OF ENTITIES SIMPLY DON'T HAVE CANDIDATES FOR THEM IN BOOTLEG! WE SHOULD ADD THEM! ############
print("Of these values that received no bootleg mention,", len(no_candidates_values), "(total)/", len(set(no_candidates_values)), "(unique), ARE NOT IN the bootleg database.")
print()
print(set(no_candidates_values))


Of these values that received no bootleg mention, 0 (total)/ 0 (unique), ARE IN the bootleg database.
Of these values that received no bootleg mention, 12 (total)/ 6 (unique), ARE NOT IN the bootleg database.

{'ladyzhenskiy', 'uasr', 'vagni', 'cerniglia', 'national energy administration', 'shoppertrak'}


#### Error rates for words

In [60]:
nomention_model_not_spanbert = []
sub_df = df_spanbert_only_errors[cols]
for index, row in sub_df.iterrows():
    qids = row['mentions']
    qids = ast.literal_eval(qids)
    words = row['example'].split(' ')
    for i in range(len(words)):
        if qids[i] == 'UNK' and words[i] not in stop_words and words[i] not in punctuation:
            nomention_model_not_spanbert.append(words[i])

print("WORSE FOR SPANBERT THAN MODEL")
ctr_nomention = Counter(nomention_model_not_spanbert).most_common(50)
print(ctr_nomention)


nomention_spanbert_or_model = []
sub_df = df_spanboot_only_errors[cols]
for index, row in sub_df.iterrows():
    qids = row['mentions']
    qids = ast.literal_eval(qids)
    words = row['example'].split(' ')
    for i in range(len(words)):
        if qids[i] == 'UNK' and words[i] not in stop_words and words[i] not in punctuation:
            nomention_spanbert_or_model.append(words[i])
            
print("WORSE FOR MODEL THAN SPANBERT")
print(Counter(nomention_spanbert_or_model).most_common(50))


WORSE FOR SPANBERT THAN MODEL
[('said', 146), ('-rrb-', 107), ('-lrb-', 90), ('years', 57), ('director', 42), ('wife', 40), ('also', 39), ('two', 37), ('research', 34), ('former', 34), ('last', 33), ('told', 33), ('church', 32), ('one', 31), ('company', 30), ('three', 29), ('according', 28), ('year', 27), ('firm', 26), ('time', 26), ('course', 24), ('says', 23), ('news', 21), ('would', 20), ('new', 19), ('1', 19), ('board', 18), ('home', 18), ('national', 18), ('living', 18), ('son', 18), ('country', 17), ('mother', 17), ('could', 17), ('people', 17), ('late', 17), ("n't", 17), ('president', 16), ('accused', 15), ('million', 15), ('head', 15), ('billion', 15), ('made', 14), ('months', 14), ('first', 14), ('van', 14), ('assouw', 14), ('times', 14), ('justice', 14), ('largest', 14)]
WORSE FOR MODEL THAN SPANBERT
[('said', 111), ('-rrb-', 51), ('-lrb-', 49), ('years', 30), ('former', 23), ('research', 21), ('also', 20), ('director', 20), ('president', 19), ('company', 18), ('one', 17), ('

In [61]:
errates_model = {}
for index, row in df_results.iterrows():
    err = 0
    if row['relation'] != row['prediction_spanboot']:
        err = 1
    words = row['example'].split(' ')
    for i in range(len(words)):
        if words[i] not in stop_words and words[i] not in punctuation:
            if words[i] in errates_model:
                errates_model[words[i]]['count'] += 1
                errates_model[words[i]]['errs'] += err
            else:
                errates_model[words[i]] = {}
                errates_model[words[i]]['count'] = 1
                errates_model[words[i]]['errs'] = err

for k, v in errates_model.items():
    errates_model[k]['errrate'] = errates_model[k]['errs']/errates_model[k]['count'] 
    if errates_model[k]['errs'] > 30:
        print(k, v)


national {'count': 1342, 'errs': 77, 'errrate': 0.05737704918032787}
director {'count': 712, 'errs': 51, 'errrate': 0.07162921348314606}
america {'count': 353, 'errs': 33, 'errrate': 0.09348441926345609}
international {'count': 459, 'errs': 48, 'errrate': 0.10457516339869281}
new {'count': 1108, 'errs': 111, 'errrate': 0.10018050541516245}
italian {'count': 271, 'errs': 44, 'errrate': 0.16236162361623616}
-lrb- {'count': 2821, 'errs': 128, 'errrate': 0.045373980857851826}
-rrb- {'count': 2821, 'errs': 128, 'errrate': 0.045373980857851826}
last {'count': 652, 'errs': 35, 'errrate': 0.05368098159509203}
said {'count': 4014, 'errs': 274, 'errrate': 0.06826108619830593}
group {'count': 411, 'errs': 37, 'errrate': 0.09002433090024331}
one {'count': 915, 'errs': 58, 'errrate': 0.0633879781420765}
police {'count': 684, 'errs': 43, 'errrate': 0.06286549707602339}
us {'count': 414, 'errs': 46, 'errrate': 0.1111111111111111}
news {'count': 515, 'errs': 40, 'errrate': 0.07766990291262135}
lange {

In [62]:
errates_spanbert = {}
for index, row in df_results.iterrows():
    err = 0
    if row['relation'] != row['prediction_spanbert']:
        err = 1
    words = row['example'].split(' ')
    for i in range(len(words)):
        if words[i] not in punctuation and words[i] not in stop_words:
            if words[i] in errates_spanbert:
                errates_spanbert[words[i]]['count'] += 1
                errates_spanbert[words[i]]['errs'] += err
            else:
                errates_spanbert[words[i]] = {}
                errates_spanbert[words[i]]['count'] = 1
                errates_spanbert[words[i]]['errs'] = err

for k, v in errates_model.items():
    if k in errates_spanbert:
        spanbert_error_ct = errates_spanbert[k]['errs']
    else:
        spanbert_error_ct = 0
    
    spanboot_error_ct = errates_model[k]['errs']
    if (spanboot_error_ct-spanbert_error_ct) >= 5 and spanboot_error_ct>8:
        print("{}: spanbert_errs {}; spanboot_errs {}".format(k, spanbert_error_ct, spanboot_error_ct))
#     elif abs(spanboot_error_ct) > 1:
#         print(errates_model[k])
#         print(errates_spanbert[k])

messina: spanbert_errs 17; spanboot_errs 22
denaro: spanbert_errs 21; spanboot_errs 26
mafia: spanbert_errs 7; spanboot_errs 15
arrest: spanbert_errs 9; spanboot_errs 16
philippines: spanbert_errs 13; spanboot_errs 19
lange: spanbert_errs 44; spanboot_errs 49
herold: spanbert_errs 5; spanboot_errs 10
child: spanbert_errs 7; spanboot_errs 13
friend: spanbert_errs 6; spanboot_errs 12
*: spanbert_errs 10; spanboot_errs 18
murder: spanbert_errs 42; spanboot_errs 54
gross: spanbert_errs 46; spanboot_errs 51
sollecito: spanbert_errs 3; spanboot_errs 9
meredith: spanbert_errs 10; spanboot_errs 15
gonzalez: spanbert_errs 10; spanboot_errs 15
attorney: spanbert_errs 15; spanboot_errs 21
governor: spanbert_errs 5; spanboot_errs 11
antara: spanbert_errs 11; spanboot_errs 16
days: spanbert_errs 16; spanboot_errs 24
june: spanbert_errs 7; spanboot_errs 12
bruno: spanbert_errs 12; spanboot_errs 20
day: spanbert_errs 17; spanboot_errs 22
kidnapping: spanbert_errs 8; spanboot_errs 21
kidnapped: spanbe

# Just Dig into Model Errors

### For models that used manual aliases, what happened on alias-examples

In [63]:
aliases_to_add = {}
aliases_to_add['Millipore'] = {'alias':'merck millipore', 'qid':'Q1669719', 'len':1}
aliases_to_add['US '] = {'alias':'united states', 'qid':'Q30', 'len':1}
aliases_to_add['U.S.'] = {'alias':'united states', 'qid':'Q30', 'len':1}
aliases_to_add['WNO'] = {'alias':"washington national opera", 'qid':'Q386613', 'len':1}

for k, v in errates_model.items():
    for alias, content in aliases_to_add.items():
        if alias.lower() in k:  
            if k in errates_spanbert:
                spanbert_error_ct = errates_spanbert[k]['errs']
            else:
                spanbert_error_ct = 0
            spanboot_error_ct = errates_model[k]['errs']
            print("{}: spanbert_errs {}; spanboot_errs {}".format(k, spanbert_error_ct, spanboot_error_ct))
            

u.s.: spanbert_errs 33; spanboot_errs 26
china-u.s.: spanbert_errs 1; spanboot_errs 0
u.s.dollars: spanbert_errs 0; spanboot_errs 0


In [64]:
for ind, row in df_errors_spanboot.iterrows():
    if ' he ' in row['example']:
        print(row[cols])
        print()

example                he formed his own company , tau , for a while , sold part to trimble , left trimble to start a gps business line at motorola -lrb- where he was also a vp -rrb- , from there to rand mcnally in chicago , and more recently to navteq .                                                                                                                                                                                                                            
relation               per:employee_of                                                                                                                                                                                                                                                                                                                                                                                                                                                    
prediction_model       no_relation                


example                she was in her early teens when her mom told her dad he could n't see his daughters if he continued taking drugs .                                                                      
relation               per:parents                                                                                                                                                                             
prediction_model       no_relation                                                                                                                                                                             
prediction_spanbert    no_relation                                                                                                                                                                             
prediction_spanboot    no_relation                                                                                                                                     

example                in the video , mario gonzalez , answering questions posed by a man off camera , says his sister protected la linea , a street gang tied to the juarez cartel , and that he acted as the liaison between the gang and attorney general 's office , collecting payoffs .                                                                                                                                                                                                                                                                         
relation               no_relation                                                                                                                                                                                                                                                                                                                                                                                                                               


example                she was in her early teens when her mom told her dad he could n't see his daughters if he continued taking drugs .                                                                      
relation               per:parents                                                                                                                                                                             
prediction_model       no_relation                                                                                                                                                                             
prediction_spanbert    no_relation                                                                                                                                                                             
prediction_spanboot    no_relation                                                                                                                                     


example                xie was the sister-in-law of wen qiang , formerly the second in command of the chongqing police and director of the justice bureau before he was arrested .                                                                                          
relation               per:employee_of                                                                                                                                                                                                                                      
prediction_model       no_relation                                                                                                                                                                                                                                          
prediction_spanbert    no_relation                                                                                                                                                              


example                no sex since july 1990 either -lrb- i was 29 -rrb- , '' he writes .                                                                                                   
relation               no_relation                                                                                                                                                           
prediction_model       no_relation                                                                                                                                                           
prediction_spanbert    no_relation                                                                                                                                                           
prediction_spanboot    per:age                                                                                                                                                               
mentions               ['No Sex (In the Champagne

In [65]:
count = 0
ids_spanboot = []
pronouns = ['he', 'she', 'her', 'his', 'him']
for index, row in df_errors_spanboot.iterrows():
    subj = ast.literal_eval(row['subj'])
    obj = ast.literal_eval(row['obj'])
    if any(s for s in subj if s in pronouns) or any(o for o in obj if o in pronouns):
        print(row[cols])
        ids_spanboot.append(row['id'])
        count += 1
        
count
      


example                she moved to sao paulo , where the baby , a boy , was born in february .                                                                                                                                        
relation               no_relation                                                                                                                                                                                                     
prediction_model       no_relation                                                                                                                                                                                                     
prediction_spanbert    no_relation                                                                                                                                                                                                     
prediction_spanboot    per:date_of_birth                                

example                he 's a manipulator and knows how to profit from the weakness of his enemies . ''                                                                       
relation               no_relation                                                                                                                                             
prediction_model       per:title                                                                                                                                               
prediction_spanbert    per:title                                                                                                                                               
prediction_spanboot    per:title                                                                                                                                               
mentions               ['UNK', 'UNK', 'UNK', 'Manipulator (device)', 'UNK', 'UNK', 'UNK', 'UNK', 'Profit (accounting)', 

Name: 4810, dtype: object
example                `` pauliina will bring a new style and flavor from her experiences as an elite player and coach in finland , and i look forward to assisting her with knowledge of the league and our team , '' rampone said .                                                                                                                                                                                                 
relation               per:countries_of_residence                                                                                                                                                                                                                                                                                                                                                                                     
prediction_model       no_relation                                                                                              

example                she filed a federal lawsuit in may against halliburton co. , its former subsidiary , kbr inc. , and others claiming she was raped by co-workers while working for a halliburton subsidiary at camp hope , baghdad , in 2005 .                                                                                                                      
relation               no_relation                                                                                                                                                                                                                                                                                                                                        
prediction_model       no_relation                                                                                                                                                                                                                                                

Name: 9952, dtype: object
example                gross studied social work at the university of maryland and the virginia commonwealth university , and he had a long career as an international development worker that took him to at least 50 countries .                                                                                                                                                                                                                                                                                                       
relation               no_relation                                                                                                                                                                                                                                                                                                                                                                                                                                  

Name: 12061, dtype: object
example                on tuesday , her driver luo xuan , 29 , who was also her lover , was sentenced to four-and-a-half years in jail for his role .                                                                               
relation               per:title                                                                                                                                                                                                    
prediction_model       no_relation                                                                                                                                                                                                  
prediction_spanbert    no_relation                                                                                                                                                                                                  
prediction_spanboot    no_relation                       

example                through it all , his allegiance to scientology had been so strong that he paid the church before the government .                                              
relation               per:religion                                                                                                                                                   
prediction_model       no_relation                                                                                                                                                    
prediction_spanbert    no_relation                                                                                                                                                    
prediction_spanboot    no_relation                                                                                                                                                    
mentions               ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'Loyalty', 'UNK', 'Sciento

example                box 53391 , san jose , california 95123 currently works out of the great oaks keller williams branch near his home .                                                                                                                                                                                          
relation               per:stateorprovinces_of_residence                                                                                                                                                                                                                                                                             
prediction_model       per:stateorprovinces_of_residence                                                                                                                                                                                                                                                                             
prediction_spanbert   

265

In [66]:
# count = 0
# ids_spanboot = []
# for index, row in df_errors_spanboot.iterrows():
#     if row['relation'] == 'per:age':
#         row_spanbert = df_errors_spanbert[row['id']].item()
#         row_spanbert['pred']
            
        
# count

In [67]:
count = 0
ids_spanbert = []
pronouns = ['he', 'she', 'her', 'his', 'him']
for index, row in df_errors_spanbert.iterrows():
    subj = ast.literal_eval(row['subj'])
    obj = ast.literal_eval(row['obj'])
    if any(s for s in subj if s in pronouns) or any(o for o in obj if o in pronouns):
        # print(row[cols])
        ids_spanbert.append(row['id'])
        count += 1
        
count
      

327

In [68]:
print(len(list(set(ids_spanboot) - set(ids_spanbert))))
print(len(list(set(ids_spanbert) - set(ids_spanboot))))

102
164


# Tacred paper insights

### For different relations, what is the error rate with and without using Bootleg embeddings?

In [94]:
count_boot = 0
for index, row in df_errors_spanboot.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count_boot += 1
        
count_bert = 0
for index, row in df_errors_spanbert.iterrows():
    if get_relations(row['subj_qids'], row['obj_qids']):
        count_bert += 1
        
print("For {}/{} errors, spanboot has *some* relation between the subj and obj\n".format(count_boot, df_errors_spanboot.shape[0]))
print("For {}/{} errors, spanbert has *some* relation between the subj and obj\n".format(count_bert, df_errors_spanbert.shape[0]))
print("Relative: ", count_bert/count_boot)

For 109/1129 errors, spanboot has *some* relation between the subj and obj

For 129/1346 errors, spanbert has *some* relation between the subj and obj

Relative:  1.18348623853211


### If subj and/or obj in tacred example has a Bootleg type, what are the error rates?

In [61]:
def get_type_ids(df):
    
    # analysis structures
    subj_type_list = []
    row_ids_subj = []
    obj_type_list = []
    row_ids_obj = []
    exists_subj = 0
    exists_obj = 0
    not_exists = 0
    
    for ind, row in df.iterrows():
        subj_qids = ast.literal_eval(row['subj_qids'])
        obj_qids = ast.literal_eval(row['obj_qids'])
        idx = row['id']

        # get the types list for the entity qid
        try:
            types_subj = du.get_qid_types(subj_qids[0], esp, typeid2typename_wd, wikidata=True)
        except:
            types_subj = []

        try:
            types_obj = du.get_qid_types(obj_qids[0], esp, typeid2typename_wd, wikidata=True)
        except:
            types_obj = []
                     
        # get the first type in the types list   
        if types_subj:
            type_subj_qid = types_subj[0] 
            if type_subj_qid in typeqid_to_id:
                type_subj_id = typeqid_to_id[type_subj_qid]+1
                subj_type_list.append(type_subj_qid)
                row_ids_subj.append(idx)
                exists_subj +=1
        else:
            type_subj_id = 0
            not_exists += 1

        if types_obj:
            type_obj_qid = types_obj[0] 
            if type_obj_qid in typeqid_to_id:
                type_obj_id = typeqid_to_id[type_obj_qid]+1
                obj_type_list.append(type_obj_qid)
                row_ids_obj.append(idx)
                exists_obj +=1
        else:
            type_obj_id = 0
            not_exists += 1

    print("The number of subjects with a type relation: ", exists_subj)
    print("The number of objects with a type relation: ",exists_obj)   
    
    return row_ids_subj, row_ids_obj

In [64]:
import json
typeqid_to_id = json.load(open("/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/bootleg_downloads/emb_data/wikidata_to_typeid_0905.json"))


In [65]:
row_ids_subj, row_ids_obj = get_type_ids(df_results_spanboot)

The number of subjects with a type relation:  8642
The number of objects with a type relation:  8600


In [66]:
subj_df = df_results[df_results['id'].isin(row_ids_subj)]

tot_subj = subj_df.shape
spanbert_errs = 0
spanboot_errs = 0
for ind, row in subj_df.iterrows():
    if row['relation'] != row['prediction_spanbert']:
        spanbert_errs += 1
    if row['relation'] != row['prediction_spanboot']:
        spanboot_errs += 1
        
print("For {}/{} subj_df examples, spanboot makes an error".format(spanboot_errs, tot_subj))
print("For {}/{} subj_df examples, spanbert makes an error".format(spanbert_errs, tot_subj))


For 726/(8642, 28) subj_df examples, spanboot makes an error
For 873/(8642, 28) subj_df examples, spanbert makes an error


In [67]:
obj_df = df_results[df_results['id'].isin(row_ids_obj)]

tot_obj = obj_df.shape
spanbert_errs = 0
spanboot_errs = 0
for ind, row in obj_df.iterrows():
    if row['relation'] != row['prediction_spanbert']:
        spanbert_errs += 1
    if row['relation'] != row['prediction_spanboot']:
        spanboot_errs += 1
        
print("For {}/{} obj_df examples, spanboot makes an error".format(spanboot_errs, tot_obj))
print("For {}/{} obj_df examples, spanbert makes an error".format(spanbert_errs, tot_obj))


For 830/(8600, 28) obj_df examples, spanboot makes an error
For 975/(8600, 28) obj_df examples, spanbert makes an error


In [68]:
# either the subj/obj/both have a type!

all_subj_and_obj_ids = row_ids_subj
all_subj_and_obj_ids.extend(row_ids_obj) 
all_subj_and_obj_ids = set(all_subj_and_obj_ids)
all_type_df = df_results[df_results['id'].isin(all_subj_and_obj_ids)]

tot_type = all_type_df.shape
spanbert_errs = 0
spanboot_errs = 0
for ind, row in all_type_df.iterrows():
    if row['relation'] != row['prediction_spanbert']:
        spanbert_errs += 1
    if row['relation'] != row['prediction_spanboot']:
        spanboot_errs += 1
        
print("For {}/{} examples with a subj/obj type, spanboot makes an error".format(spanboot_errs, tot_type))
print("For {}/{} examples with a subj/obj type, spanbert makes an error".format(spanbert_errs, tot_type))
print("Error Rate SpanBERT/SpanBOOT = {}".format((spanbert_errs/tot_type[0])/(spanboot_errs/tot_type[0])))

For 1010/(12044, 28) examples with a subj/obj type, spanboot makes an error
For 1211/(12044, 28) examples with a subj/obj type, spanbert makes an error
Error Rate SpanBERT/SpanBOOT = 1.199009900990099


In [109]:
# either subj/obj have a entity

tot_count = 0
spanbert_errs = 0
spanboot_errs = 0
for ind, row in df_results.iterrows():
    subj =  ast.literal_eval(row['subj_qids'])
    obj = ast.literal_eval(row['obj_qids'])
    if (subj and subj[0] != 'UNK' ) or ( obj and obj[0] != 'UNK' ):
        if row['relation'] != row['prediction_spanbert']:
            spanbert_errs += 1
        if row['relation'] != row['prediction_spanboot']:
            spanboot_errs += 1
        tot_count += 1
        
print("For {}/{} obj_df examples, spanboot makes an error".format(spanboot_errs, tot_count))
print("For {}/{} obj_df examples, spanbert makes an error".format(spanbert_errs, tot_count))

For 1038/12621 obj_df examples, spanboot makes an error
For 1248/12621 obj_df examples, spanbert makes an error


In [110]:
print(spanbert_errs/spanboot_errs)

1.2023121387283238


### Alternate Names

In [98]:
#SPANBOOT
count_boot_errors = 0
relations_errs_matchqids = []
sub_df_boot = df_errors_spanboot[df_errors_spanboot['relation'] == 'org:alternate_names']
sub_df_bert = df_errors_spanbert[df_errors_spanbert['relation'] == 'org:alternate_names']
sub_df_total = df_results[df_results['relation'] == 'org:alternate_names']

print("Total Alternate Names ", sub_df_total.shape)
print("BOOT errors: ", sub_df_boot.shape)
print("BERT errors: ", sub_df_bert.shape)

print(sub_df_bert.shape[0]/sub_df_boot.shape[0])
sub_df_bert[['example', 'subj_mentions', 'obj_mentions']]

Total Alternate Names  (245, 28)
BOOT errors:  (19, 28)
BERT errors:  (26, 28)
1.368421052631579


,example,subj_mentions,obj_mentions
1142,"beijing , dec. 14 -lrb- xinhua -rrb- -- china 's power consumption in november rose 27.63 percent year on year to 328.39 billion kilowatt-hours -lrb- kwh -rrb- , up for the sixth consecutive month since june , china 's national energy administration -lrb- nea -rrb- announced monday .",['Nuclear Energy Agency'],"['UNK', 'UNK', 'UNK']"
1467,"u.s. immigration and customs enforcement agents seized documents and other materials at the koch foods plant and at koch foods inc. 's chicago area headquarters , said brian moskowitz , a special agent for ice enforcement .","['Koch Foods', 'Koch Foods']","['Koch Foods', 'Koch Foods', 'UNK']"
1505,poverty and social impact analysis of reforms : lessons and examples from implementation http://siteresources.worldbank.org/intpsia/resources/brochure_psiales ... poverty and social impact analysis of reforms presents a collection of case studies that illustrate the spectrum of sectors and policy reforms to which psia can be applied ; it also elaborates on the broad range of analytical tools and techniques that can be used for psia .,['Professional Ski Instructors of America &amp; American Association of Snowboard Instructors'],"['Poverty', 'UNK', 'UNK', 'Change impact analysis', 'Change impact analysis', 'UNK', 'UNK']"
2929,"the baron 's cup 2010 horse racing event , co-organized by the myanmar equestrian federation -lrb- mef -rrb- and myanmar brewery ltd , will be held this month to boost the country 's equestrian sport , according to the mef friday .","['Myanmar', 'Equestrianism', 'UNK']",['MEF Forum']
4037,"twenty percent of those surveyed in 10 countries by visa asia pacific and pata , the pacific asia travel association , named thailand as their most likely destination , visa and bangkok-based pata said in a statement .",['Pacific Asia Travel Association'],"['Pacific Asia Travel Association', 'Pacific Asia Travel Association', 'Pacific Asia Travel Association', 'Pacific Asia Travel Association']"
4234,"the two projects -- a trachoma prevention plan and a cooking oil plan -- are jointly organized by the new york-based helen keller international -lrb- hki -rrb- , the united nations children 's fund and the world health organization , the spokesman said , adding that the hki will implement the two programs using funds donated by taiwan .",['Helen Keller International'],"['Helen Keller International', 'Helen Keller International', 'Helen Keller International']"
4467,"the 2007 international music festival organized by national taiwan symphony orchestra -lrb- ntso -rrb- will begin oct. 6 in taichung city in central taiwan , an ntso spokesman said tuesday .","['National Taiwan Symphony Orchestra', 'National Taiwan Symphony Orchestra', 'National Taiwan Symphony Orchestra', 'National Taiwan Symphony Orchestra']",['UNK']
4493,"before the reform and opening-up drive started in the late 1970s , political faith was the sole motive that secured local officials ' adherence to central authorities ' decisions , but now they put local economic interests above all , said wang xiaoguang , head of the economic operation and development section of the research institute of economy under the national development and reform commission -lrb- ndrc -rrb- .",['National Development and Reform Commission'],"['National Development and Reform Commission', 'National Development and Reform Commission', 'National Development and Reform Commission', 'National Development and Reform Commission', 'National Development and Reform Commission']"
5300,"this was the fifth year of australia 's involvement in pacific partnership , with australian defense force -lrb- adf -rrb- personnel assisting in aid activities in vietnam , cambodia , indonesia , papua new guinea and timor-leste .",['Australian Defence Force'],"['Australian Defence Force', 'Australian Defence Force']"
7787,"the two projects -- a trachoma prevention plan and a cooking oil plan -- are jointly organized by the new york-based hele

### error rates by number of entities in the example

In [70]:
import statistics
def split_by_median(dict_of_errors, median):
    lt_median_count = 0
    mt_median_count = 0
    lt_median_error = 0
    mt_median_error = 0
    for k, v in dict_of_errors.items():
        if k <= median:
            lt_median_count += v['count']
            lt_median_error += v['error']
        else:
            mt_median_count += v['count']
            mt_median_error += v['error']
    print("Counts: ", lt_median_count, mt_median_count)
    print("Errors: ", lt_median_error, mt_median_error)
    lower_err_rate = lt_median_error/lt_median_count
    upper_err_rate = mt_median_error/mt_median_count
    print("Error Rates: ", lower_err_rate, upper_err_rate)
    
    lt_median_len = []
    mt_median_len = []
    for k, v in dict_of_errors.items():
        if k < median:
            lt_median_len.append(v['len'])
        else:
            mt_median_len.append(v['len'])
            
    print(sum(lt_median_len)/lt_median_count, sum(mt_median_len)/mt_median_count)
    return lower_err_rate, upper_err_rate
    

In [71]:
dict_of_errors_spanbert = {}
num_mentions_spanbert = []

dict_of_errors_spanboot = {}
num_mentions_spanboot = []

for index, row in df_results.iterrows():
    words = row['example'].split(' ')
    word_count = len([word for word in words if word not in stop_words and word not in punctuation])
    mentions = ast.literal_eval(row['qids'])
    non_unk_mentions = len([ment for ment in mentions if ment != 'UNK' and ment != 'NF']) #/word_count
    subj_qids = row['subj_qids']
    obj_qids = row['obj_qids']
    
    if non_unk_mentions > 0:
        err = 0
        if row['relation'] != row['prediction_spanbert']:
            err = 1
        if non_unk_mentions in dict_of_errors_spanbert:
            dict_of_errors_spanbert[non_unk_mentions]['count'] += 1
            dict_of_errors_spanbert[non_unk_mentions]['error'] += err
            dict_of_errors_spanbert[non_unk_mentions]['len'] += len(words)
        else:
            dict_of_errors_spanbert[non_unk_mentions] = {}
            dict_of_errors_spanbert[non_unk_mentions]['count'] = 1
            dict_of_errors_spanbert[non_unk_mentions]['error'] = err
            dict_of_errors_spanbert[non_unk_mentions]['len'] = len(words)

        num_mentions_spanbert.append(non_unk_mentions)

        err = 0
        if row['relation'] != row['prediction_spanboot']:
            err = 1
        if non_unk_mentions in dict_of_errors_spanboot:
            dict_of_errors_spanboot[non_unk_mentions]['count'] += 1
            dict_of_errors_spanboot[non_unk_mentions]['error'] += err
            dict_of_errors_spanboot[non_unk_mentions]['len'] += len(words)
        else:
            dict_of_errors_spanboot[non_unk_mentions] = {}
            dict_of_errors_spanboot[non_unk_mentions]['count'] = 1
            dict_of_errors_spanboot[non_unk_mentions]['error'] = err
            dict_of_errors_spanboot[non_unk_mentions]['len'] = len(words)
        num_mentions_spanboot.append(non_unk_mentions)


In [72]:
median_boot = statistics.median(num_mentions_spanboot)
lower_boot, upper_boot = split_by_median(dict_of_errors_spanboot, median_boot-1)
print("MEDIAN: ", median_boot)

Counts:  7343 7980
Errors:  460 667
Error Rates:  0.06264469562848972 0.08358395989974937
22.159335421489853 46.7531328320802
MEDIAN:  8


In [73]:
median_bert = statistics.median(num_mentions_spanbert)
lower_bert, upper_bert = split_by_median(dict_of_errors_spanbert, median_bert-1)
print("MEDIAN: ", median_bert)

Counts:  7343 7980
Errors:  558 784
Error Rates:  0.07599073947977666 0.09824561403508772
22.159335421489853 46.7531328320802
MEDIAN:  8


In [74]:
print(7343+7980)

15323


In [75]:
print("The error gap between the spanbert and spanboot model is 16% lower on the slice with an above-median number of entities.")
print((upper_bert-upper_boot)/(lower_bert-lower_boot))

The error gap between the spanbert and spanboot model is 16% lower on the slice with an above-median number of entities.
1.0985767991407085


### Now for number of relations

In [76]:
rel_errors_spanbert = {}
num_rels_spanbert = []

rel_errors_spanboot = {}
num_rels_spanboot = []

for index, row in df_results.iterrows():
    qids = ast.literal_eval(row['qids'])
    
    rels_count = 0
    total_pairs = 0
    for i in range(len(qids)):
        if qids[i] != 'UNK':
            for j in range(len(qids)):
                if i < j and qids[j] != 'UNK':
                    rels = [esp.get_relation_name(r) for r in esp.get_all_relations(qids[i], qids[j])]
                    if rels:
                        rels_count += 1 #len(rels)
                    total_pairs += 1
    if rels_count > 0:
        #print(rels_count)
        err = 0
        if row['relation'] != row['prediction_spanbert']:
            err = 1
        if rels_count in rel_errors_spanbert:
            rel_errors_spanbert[rels_count]['count'] += 1
            rel_errors_spanbert[rels_count]['error'] += err
            rel_errors_spanbert[rels_count]['len'] += total_pairs
        else:
            rel_errors_spanbert[rels_count] = {}
            rel_errors_spanbert[rels_count]['count'] = 1
            rel_errors_spanbert[rels_count]['error'] = err
            rel_errors_spanbert[rels_count]['len'] = total_pairs
        num_rels_spanbert.append(rels_count)

        err = 0
        if row['relation'] != row['prediction_spanboot']:
            err = 1
        if rels_count in rel_errors_spanboot:
            rel_errors_spanboot[rels_count]['count'] += 1
            rel_errors_spanboot[rels_count]['error'] += err
            rel_errors_spanboot[rels_count]['len'] += total_pairs
        else:
            rel_errors_spanboot[rels_count] = {}
            rel_errors_spanboot[rels_count]['count'] = 1
            rel_errors_spanboot[rels_count]['error'] = err
            rel_errors_spanboot[rels_count]['len'] = total_pairs
        num_rels_spanboot.append(rels_count)
    

In [99]:
2448 + 2956

5404

In [78]:
median_boot = statistics.median(num_rels_spanboot)
lower_boot, upper_boot = split_by_median(rel_errors_spanboot, median_boot-1)
print(median_boot)

Counts:  2448 2956
Errors:  221 254
Error Rates:  0.09027777777777778 0.08592692828146144
39.25816993464052 115.36941813261164
4.0


In [79]:
median_bert = statistics.median(num_rels_spanbert)
lower_bert, upper_bert = split_by_median(rel_errors_spanbert, median_bert-1)
print(median_bert)

Counts:  2448 2956
Errors:  232 316
Error Rates:  0.09477124183006536 0.10690121786197564
39.25816993464052 115.36941813261164
4.0


In [80]:
print((upper_bert-upper_boot)/(lower_bert-lower_boot))

4.6677328084635255


### For number of types in the example

In [81]:
type_errors_spanbert = {}
num_types_spanbert = []

type_errors_spanboot = {}
num_types_spanboot = []

for ind, row in df_results.iterrows():
    qids = ast.literal_eval(row['qids'])
    num_types = 0
    for qid in qids:
        try:
            types = du.get_qid_types(qid, esp, typeid2typename_wd_0905, wikidata=True)
        except:
            types= [] 
        if types:
            num_types += 1 #len(types)
     
    #print(num_types)
    if num_types > -1: 
        err = 0
        if row['relation'] != row['prediction_spanbert']:
            err = 1
        if num_types in type_errors_spanbert:
            type_errors_spanbert[num_types]['count'] += 1
            type_errors_spanbert[num_types]['error'] += err
            type_errors_spanbert[num_types]['len'] += len(qids)
        else:
            type_errors_spanbert[num_types] = {}
            type_errors_spanbert[num_types]['count'] = 1
            type_errors_spanbert[num_types]['error'] = err
            type_errors_spanbert[num_types]['len'] = len(qids)
        num_types_spanbert.append(num_types)

        err = 0
        if row['relation'] != row['prediction_spanboot']:
            err = 1
        if num_types in type_errors_spanboot:
            type_errors_spanboot[num_types]['count'] += 1
            type_errors_spanboot[num_types]['error'] += err
            type_errors_spanboot[num_types]['len'] += len(qids)
        else:
            type_errors_spanboot[num_types] = {}
            type_errors_spanboot[num_types]['count'] = 1
            type_errors_spanboot[num_types]['error'] = err
            type_errors_spanboot[num_types]['len'] = len(qids)
        num_types_spanboot.append(num_types)
    
    

In [82]:
median_boot = statistics.median(num_types_spanboot)
print("MEDIAN:", median_boot)
lower_boot, upper_boot = split_by_median(type_errors_spanboot, median_boot-1)


MEDIAN: 7
Counts:  7052 8457
Errors:  432 697
Error Rates:  0.061259217243335225 0.08241693271845808
21.320476460578558 45.95684048717039


In [83]:
median_bert = statistics.median(num_types_spanbert)
print("MEDIAN:", median_bert)
lower_bert, upper_bert  = split_by_median(type_errors_spanbert, median_bert-1)

MEDIAN: 7
Counts:  7052 8457
Errors:  515 831
Error Rates:  0.07302892796369824 0.09826179496275275
21.320476460578558 45.95684048717039


In [84]:
print((upper_bert-upper_boot)/(lower_bert-lower_boot))

1.3462405849007955


In [100]:
7052 + 8457

15509

## look at the specific type or relation 

In [103]:
# relations between the subject and object

dict_bootleg_rel_by_tacred_rel = {}

for ind, row in df_results.iterrows():
    bootleg_relations = []
    rels = get_relations(row['subj_qids'], row['obj_qids'])
    bootleg_relations.append(rels)
                    
    tacred_relation = row['relation']
    spanboot_err = 0
    spanbert_err = 0
    if row['relation'] != row['prediction_spanboot']:
        spanboot_err = 1
    if row['relation'] != row['prediction_spanbert']:
        spanbert_err = 1
    
    if rels:
        if tacred_relation in dict_bootleg_rel_by_tacred_rel:
            dict_bootleg_rel_by_tacred_rel[tacred_relation]['rels'].append(bootleg_relations)  
            dict_bootleg_rel_by_tacred_rel[tacred_relation]['boot_errs'] += spanboot_err
            dict_bootleg_rel_by_tacred_rel[tacred_relation]['bert_errs'] += spanbert_err
        else:
            dict_bootleg_rel_by_tacred_rel[tacred_relation] = {}
            dict_bootleg_rel_by_tacred_rel[tacred_relation]['rels'] = [bootleg_relations]
            dict_bootleg_rel_by_tacred_rel[tacred_relation]['boot_errs'] = spanboot_err
            dict_bootleg_rel_by_tacred_rel[tacred_relation]['bert_errs'] = spanbert_err
    

In [104]:
print("KEY ", "NUM RELS ", "BOOT ERRS ", "BERT ERRS ")
for k,v in dict_bootleg_rel_by_tacred_rel.items():
    print(k, len(v['rels']), v['boot_errs'], v['bert_errs'])

KEY  NUM RELS  BOOT ERRS  BERT ERRS 
no_relation 274 48 60
per:countries_of_residence 26 3 5
org:country_of_headquarters 36 18 19
org:parents 9 6 6
per:stateorprovince_of_death 2 0 0
per:origin 55 15 13
org:top_members/employees 14 1 2
per:title 13 3 3
per:cities_of_residence 29 1 4
per:employee_of 29 3 6
org:shareholders 1 0 0
org:stateorprovince_of_headquarters 10 0 4
org:city_of_headquarters 19 1 2
per:spouse 7 1 0
per:city_of_birth 2 0 0
org:founded_by 6 2 0
per:schools_attended 4 3 2
per:country_of_death 1 1 1
per:stateorprovinces_of_residence 4 3 2
per:city_of_death 1 0 0


In [108]:
key = 'org:stateorprovince_of_headquarters'
dict_bootleg_rel_by_tacred_rel[key]
# sub_df_key = df_results[df_results['relation'] == key]
# print(sub_df_key.shape[0])

{'rels': [[['located in the administrative territorial entity']],
  [['located in the administrative territorial entity']],
  [['headquarters location']],
  [['located in the administrative territorial entity']],
  [['located in the administrative territorial entity']],
  [['located in the administrative territorial entity']],
  [['headquarters location']],
  [['located in the administrative territorial entity']],
  [['located in the administrative territorial entity']],
  [['headquarters location']]],
 'boot_errs': 0,
 'bert_errs': 4}

In [107]:
key = 'per:cities_of_residence'
dict_bootleg_rel_by_tacred_rel[key]

{'rels': [[['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['work location']],
  [['place of birth']],
  [['work location']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['work location']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']],
  [['place of birth']]],
 'boot_errs': 1,
 'bert_errs': 4}

In [90]:
# # relations anywhere in the example

# dict_bootleg_rel_by_tacred_rel = {}

# for ind, row in df_results.iterrows():
#     qids = ast.literal_eval(row['qids'])
    
#     rels_count = 0
#     total_pairs = 0
#     for i in range(len(qids)):
#         bootleg_relations = []
#         if qids[i] != 'UNK':
#             for j in range(len(qids)):
#                 if i < j and qids[j] != 'UNK':
#                     rels = [esp.get_relation_name(r) for r in esp.get_all_relations(qids[i], qids[j])]
#                     if rels:
#                         print(rels)
#                         bootleg_relations.append(rels)
#                     total_pairs += 1
                    
#         tacred_relation = row['relation']
                    
#         if tacred_relation in dict_bootleg_rel_by_tacred_rel:
#             dict_bootleg_rel_by_tacred_rel[tacred_relation].extend(bootleg_relations)
            
#         else:
#             dict_bootleg_rel_by_tacred_rel[tacred_relation] = bootleg_relations
# #     if rels_count > 0:
# #         #print(rels_count)
# #         err = 0
# #         if row['relation'] != row['prediction_spanbert']:
# #             err = 1
# #         if rels_count in rel_errors_spanbert:  
# #         else: 
# #         num_rels_spanbert.append(rels_count)

# #         err = 0
# #         if row['relation'] != row['prediction_spanboot']:
# #             err = 1
# #         if rels_count in rel_errors_spanboot:
# #         else:  
# #         num_rels_spanboot.append(rels_count)

    
    

### conditioning on the train-set popularity of a subj/obj

In [ ]:
with open('/dfs/scratch1/simran/bootleg_downstream/error_outputs/subj_counts.json', 'r') as fp:
    train_subj_counts = json.load(fp)
    
with open('/dfs/scratch1/simran/bootleg_downstream/error_outputs/obj_counts.json', 'r') as fp:
    train_obj_counts = json.load(fp)

In [ ]:
train_subj_counts = {k: v for k, v in sorted(train_subj_counts.items(), key=lambda item: item[1], reverse=True)}
print(len(train_subj_counts))

In [ ]:
train_obj_counts = {k: v for k, v in sorted(train_obj_counts.items(), key=lambda item: item[1], reverse=True)}
print(len(train_obj_counts))

In [130]:
# import matplotlib.pylab as plt
# plt.figure()
# lists = subj_results_dict.items() # return a list of tuples
# x, y = zip(*lists) # unpack a list of pairs into two tuples
# plt.plot(x, y)
# plt.show()

In [131]:
subj_spanboot = {}
subj_spanbert = {}
obj_spanboot = {}
obj_spanbert = {}

from tqdm import tqdm
for ind, row in tqdm(df_results.iterrows()):
    
    subj = ' '.join(ast.literal_eval(row['subj']))
    obj = ' '.join(ast.literal_eval(row['obj']))
    
    # boot
    if subj in subj_spanboot:
        subj_spanboot[subj]['count'] = subj_spanboot[subj]['count'] + 1
    else:
        subj_spanboot[subj] = {}
        subj_spanboot[subj]['errs'] = 0
        subj_spanboot[subj]['count'] = 1
    
    if obj in obj_spanboot:
        obj_spanboot[obj]['count'] = obj_spanboot[obj]['count'] + 1
    else:
        obj_spanboot[obj] = {}
        obj_spanboot[obj]['errs'] = 0
        obj_spanboot[obj]['count'] = 1
    
    if row['relation'] != row['prediction_spanboot']:
        subj_spanboot[subj]['errs'] = subj_spanboot[subj]['errs'] + 1
        obj_spanboot[obj]['errs'] = obj_spanboot[obj]['errs'] + 1
           
            
     # bert
    if subj in subj_spanbert:
        subj_spanbert[subj]['count'] = subj_spanbert[subj]['count'] + 1
    else:
        subj_spanbert[subj] = {}
        subj_spanbert[subj]['errs'] = 0
        subj_spanbert[subj]['count'] = 1
    
    if obj in obj_spanbert:
        obj_spanbert[obj]['count'] = obj_spanbert[obj]['count'] + 1
    else:
        obj_spanbert[obj] = {}
        obj_spanbert[obj]['errs'] = 0
        obj_spanbert[obj]['count'] = 1
        
    if row['relation'] != row['prediction_spanbert']:
        subj_spanbert[subj]['errs'] = subj_spanbert[subj]['errs'] + 1   
        obj_spanbert[obj]['errs'] = obj_spanbert[obj]['errs'] + 1


15509it [00:03, 4602.82it/s]


In [132]:
print(subj_spanboot)
print(len(subj_spanboot))

{'progressive democrats of america': {'errs': 10, 'count': 106}, 'his': {'errs': 72, 'count': 1318}, 'eugenio vagni': {'errs': 9, 'count': 101}, 'he': {'errs': 66, 'count': 1420}, 'pacific asia travel association': {'errs': 7, 'count': 110}, 'helen keller international': {'errs': 1, 'count': 26}, 'adf': {'errs': 11, 'count': 232}, 'him': {'errs': 4, 'count': 300}, 'laura silsby': {'errs': 7, 'count': 98}, 'richard gage': {'errs': 5, 'count': 129}, 'her': {'errs': 55, 'count': 890}, 'ilan ramon': {'errs': 4, 'count': 52}, 'ramon': {'errs': 1, 'count': 12}, 'nra': {'errs': 2, 'count': 34}, 'national restaurant association': {'errs': 8, 'count': 211}, 'norris church': {'errs': 5, 'count': 80}, 'countrywide financial': {'errs': 7, 'count': 42}, 'corporate library': {'errs': 36, 'count': 247}, 'alico': {'errs': 34, 'count': 232}, 'heloise': {'errs': 7, 'count': 112}, 'anna mae pictou aquash': {'errs': 2, 'count': 13}, 'craig ranke': {'errs': 2, 'count': 45}, 'national taiwan symphony orches

In [133]:
subj_results_dict = {}
train_subj_counts_lst = []
for k, v in subj_spanboot.items():
    subj_results_dict[k] = {}
    
    if k in train_subj_counts:
        subj_results_dict[k]['train_count'] = train_subj_counts[k]
    else:
         subj_results_dict[k]['train_count'] = 0
    train_subj_counts_lst.append(subj_results_dict[k]['train_count'])
            
    subj_results_dict[k]['spanboot_errate'] = subj_spanboot[k]['errs']/subj_spanboot[k]['count']
    subj_results_dict[k]['spanbert_errate'] = subj_spanbert[k]['errs']/subj_spanbert[k]['count']


NameError: name 'train_subj_counts' is not defined

In [ ]:
import statistics
mean = statistics.mean(train_subj_counts_lst)
print(mean)

In [ ]:
# "HEAD"

boot_tot = 0
bert_tot = 0
tot_count = 0

for k, v in subj_results_dict.items():
    if v['train_count'] > mean:
        boot_tot += v['spanboot_errate']
        bert_tot += v['spanbert_errate']
        tot_count += 1
        
# print(boot_tot/tot_count)
# print(bert_tot/tot_count)
gt_mean_diff = bert_tot/tot_count - boot_tot/tot_count
print(abs(gt_mean_diff)*100) 
print(tot_count)

In [77]:
# TAIL

boot_tot = 0
bert_tot = 0
tot_count = 0

for k, v in subj_results_dict.items():
    if v['train_count'] <= mean:
        boot_tot += v['spanboot_errate']
        bert_tot += v['spanbert_errate']
        tot_count += 1
        
# print(boot_tot/tot_count)
# print(bert_tot/tot_count)
lt_mean_diff = bert_tot/tot_count - boot_tot/tot_count
print(lt_mean_diff*100) 
print(tot_count)

NameError: name 'subj_results_dict' is not defined

In [78]:
print("The difference in the (BERT-BOOT error rate) is {}x bigger below-mean train-occurrences than above-mean.".format(lt_mean_diff/gt_mean_diff))

NameError: name 'lt_mean_diff' is not defined